### 基本库import

In [1]:
#-*- encoding:utf-8 -*-
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import f1_score, accuracy_score, log_loss
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
%matplotlib inline

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
import lightgbm as lgb
from sklearn import metrics

D:\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.



## 获取feature

In [2]:
train = pd.read_csv('./data/train_featureV1.csv')
test = pd.read_csv('./data/test_featureV1.csv')

In [3]:
train = train.sample(frac=1)  
train.head()

,uid,label,voice_opp_num_unique_count,voice_opp_num_count,voice_opp_head_unique_count,voice_opp_len_3,voice_opp_len_5,voice_opp_len_6,voice_opp_len_7,voice_opp_len_8,...,"('w_a_downflow_max', 0.0)","('w_a_downflow_max', 1.0)","('w_a_downflow_min', 0.0)","('w_a_downflow_min', 1.0)","('w_a_downflow_median', 0.0)","('w_a_downflow_median', 1.0)","('w_a_downflow_mean', 0.0)","('w_a_downflow_mean', 1.0)","('w_a_downflow_sum', 0.0)","('w_a_downflow_sum', 1.0)"
4405,u4406,1,48.0,120.0,39.0,0.0,8.0,0.0,0.0,3.0,...,32884193.0,29869689.0,0.0,0.0,8178.0,16136.0,2.278208e+05,7.828379e+05,392762983.0,320963542.0
4331,u4332,1,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,234506317.0,34132.0,60.0,0.0,70350.0,14470.0,2.009716e+07,1.576800e+04,864177816.0,63072.0
1,u0002,0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,15438.0,0.0,0.0,0.0,1656.0,0.0,3.267207e+03,0.000000e+00,94749.0,0.0
849,u0850,0,25.0,115.0,16.0,0.0,1.0,0.0,0.0,0.0,...,40507314.0,52066905.0,0.0,40.0,856.0,6806.0,6.292996e+05,1.217871e+06,200117261.0,109608385.0
1009,u1010,0,28.0,281.0,24.0,0.0,1.0,0.0,1.0,0.0,...,3429.0,0.0,144.0,0.0,1786.5,0.0,1.786500e+03,0.000000e+00,3573.0,0.0


## ensembling

In [4]:
# Some useful parameters which will come in handy later on
ntrain = train.shape[0]
ntest = test.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 3 # set folds for out-of-fold prediction
kf = KFold(ntrain, n_folds= NFOLDS, random_state=SEED)

# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)
    
# Class to extend XGboost classifer

### 设置模型参数

In [70]:
# Put in our parameters for said classifiers
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 300,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 8,
   # 'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 0
}

# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators':300,
    #'max_features': 0.5,
    'max_depth': 8,
    #'min_samples_leaf': 2,
    'verbose': 0
}

# AdaBoost parameters
ada_params = {
    'n_estimators': 300,
    'learning_rate' : 0.75
}

# Gradient Boosting parameters
gb_params = {
    'n_estimators': 300,
    # 'max_features': 0.2,
    'max_depth': 8,
   # 'min_samples_leaf': 2,
    'verbose': 0
}

# Support Vector Classifier parameters 
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025
    }

In [71]:
# 实例化模型
# Create 5 objects that represent our 4 models
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)
svc = SklearnHelper(clf=SVC, seed=SEED, params=svc_params)

#准备训练测试集
# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
y_train = train['label'].ravel()
x_train = train.drop(['uid','label'], axis=1).values # Creates an array of the train data
x_test = test.drop(['uid'], axis=1).values # Creats an array of the test data


In [72]:
# 防止过拟合 划分训练测试集
#def get_oof(clf, x_train, y_train, x_test):
#    clf.train(x_train, y_train)
#
#    oof_train = clf.predict(x_train)
#    oof_test = clf.predict(x_test)
#    
#    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

# 防止过拟合 划分训练测试集
def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [73]:
# Create our OOF train and test predictions. These base results will be used as new features
et_oof_train, et_oof_test = get_oof(et, x_train, y_train, x_test) # Extra Trees

print("Training is complete")

Training is complete


In [74]:
rf_oof_train, rf_oof_test = get_oof(rf,x_train, y_train, x_test) # Random Forest

print("Training is complete")

D:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:305: UserWarning:

Warm-start fitting without increasing n_estimators does not fit new trees.



Training is complete


In [75]:
ada_oof_train, ada_oof_test = get_oof(ada, x_train, y_train, x_test) # AdaBoost

print("Training is complete")

Training is complete


In [76]:
gb_oof_train, gb_oof_test = get_oof(gb,x_train, y_train, x_test) # Gradient Boost

print("Training is complete")

Training is complete


In [77]:
#svc_oof_train, svc_oof_test = get_oof(svc,x_train, y_train, x_test) # Support Vector Classifier

print("Training is complete")

Training is complete


In [78]:
# 得出特征重要性评估
rf_feature = rf.feature_importances(x_train,y_train)
et_feature = et.feature_importances(x_train, y_train)
ada_feature = ada.feature_importances(x_train, y_train)
#gb_feature = gb.feature_importances(x_train,y_train)
#svc_feature = svc.feature_importances(x_train,y_train)

[1.87397995e-02 1.39775049e-02 1.56938296e-02 6.15061265e-04
 4.65879554e-03 3.94195511e-04 1.20172072e-03 4.62949777e-03
 1.26383853e-03 1.43272018e-03 1.69518908e-02 6.88223949e-02
 7.87550510e-04 6.19051878e-04 4.49405154e-03 2.22346477e-05
 0.00000000e+00 0.00000000e+00 4.03227799e-05 2.06498013e-05
 0.00000000e+00 0.00000000e+00 2.22952695e-02 8.78444410e-03
 6.83210127e-03 0.00000000e+00 2.66912939e-04 1.42427675e-02
 2.70405853e-02 1.10520615e-02 1.45007808e-02 6.69771161e-03
 8.78252391e-03 1.12157058e-02 1.43357089e-02 2.06943265e-02
 2.39291626e-02 1.21644123e-02 2.29182032e-02 1.46330989e-02
 8.32991154e-03 6.13321227e-03 0.00000000e+00 1.92094273e-04
 2.36960449e-02 1.01165038e-02 6.54853708e-03 1.26138571e-04
 0.00000000e+00 9.49844952e-06 8.58549454e-04 1.92763146e-04
 3.17422503e-03 8.67573659e-03 3.62271050e-02 1.35667409e-03
 1.42933579e-02 2.96940341e-03 7.55151021e-02 8.29931051e-04
 4.77383909e-04 7.96357784e-05 1.68596133e-04 6.55829901e-05
 5.53853306e-05 8.179527

D:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:305: UserWarning:

Warm-start fitting without increasing n_estimators does not fit new trees.



[9.96743543e-03 2.29825320e-02 4.82992007e-02 2.52837855e-03
 2.84821000e-03 1.52603287e-03 2.54667128e-03 3.11989832e-03
 6.36738723e-03 3.06840895e-03 2.23688038e-02 2.18941707e-02
 1.30980253e-03 1.53974829e-03 1.07066259e-02 2.85787278e-04
 0.00000000e+00 0.00000000e+00 3.29141749e-04 1.01102026e-04
 0.00000000e+00 0.00000000e+00 3.20092876e-02 1.12967391e-02
 3.48500488e-03 3.99648280e-05 2.97195878e-04 1.49834091e-02
 3.74105409e-02 9.72741864e-03 6.94917594e-03 4.19559543e-03
 6.13671568e-03 1.02925590e-02 6.63433088e-03 1.21781164e-02
 2.15794303e-02 1.40585585e-02 3.11882948e-02 8.94910116e-03
 5.73244626e-03 4.59034188e-03 1.82307050e-05 5.63657103e-04
 4.08152419e-02 7.92777206e-03 9.93463830e-03 1.40225567e-04
 0.00000000e+00 8.37357768e-04 3.83275779e-03 3.97727835e-04
 2.21708475e-03 1.33719495e-02 5.89346857e-02 3.30789982e-03
 1.18454411e-02 8.29728178e-03 1.39040199e-02 2.12701215e-03
 9.84980069e-04 6.07116921e-04 1.70043946e-04 7.29400763e-04
 1.28537539e-04 3.088376

## 第二层模型

In [79]:
base_predictions_train = pd.DataFrame( {
    'RandomForest': rf_oof_train.ravel(),
     'ExtraTrees': et_oof_train.ravel(),
     'AdaBoost': ada_oof_train.ravel(),
     'GradientBoost': gb_oof_train.ravel(),
     'label':y_train.ravel()
    })
base_predictions_train.describe()

,AdaBoost,ExtraTrees,GradientBoost,RandomForest,label
count,4999.000000,4999.000000,4999.000000,4999.000000,4999.000000
mean,0.138628,0.011802,0.121024,0.061412,0.180036
std,0.345592,0.108006,0.326188,0.240109,0.384256
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [80]:
# 可视化第一层模型的相关程度
data = [
    go.Heatmap(
        z= base_predictions_train.astype(float).corr().values ,
        x= base_predictions_train.columns.values,
        y= base_predictions_train.columns.values,
          colorscale='Viridis',
            showscale=True,
            reversescale = True
    )
]

py.iplot(data, filename='labelled-heatmap')

In [81]:
# 生成第二层模型的训练测试集
#x_train = np.concatenate(( et_oof_train, rf_oof_train, ada_oof_train,gb_oof_train, svc_oof_train), axis=1)
#x_test = np.concatenate(( et_oof_test, rf_oof_test, ada_oof_test,gb_oof_test, svc_oof_test), axis=1)
x_train = np.concatenate(( et_oof_train, ada_oof_train,rf_oof_train, gb_oof_train), axis=1)
x_test = np.concatenate(( et_oof_test, ada_oof_test,rf_oof_test, gb_oof_test), axis=1)

In [82]:
x_train

array([[0., 0., 0., 0.],
       [0., 1., 1., 1.],
       [0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [83]:
y_train

array([1, 1, 0, ..., 0, 0, 0], dtype=int64)

## lgboost 第二层分类器

In [84]:
dtrain = lgb.Dataset(x_train,label=y_train)
dtest = lgb.Dataset(x_test)

In [85]:
lgb_params =  {
    'boosting_type': 'gbdt',
    'objective': 'binary',
#    'metric': ('multi_logloss', 'multi_error'),
    #'metric_freq': 100,
    'is_training_metric': False,
    'min_data_in_leaf': 12,
    'num_leaves': 64,
    'learning_rate': 0.08,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'verbosity':-1,
    'max_depth':7,
#    'gpu_device_id':2,
#    'device':'gpu'
#    'lambda_l1': 0.001,
#    'skip_drop': 0.95,
#    'max_drop' : 10
    #'lambda_l2': 0.005
    #'num_threads': 18
}    

In [86]:
def evalMetric(preds,dtrain):
    
    label = dtrain.get_label()
    
    
    pre = pd.DataFrame({'preds':preds,'label':label})
    pre= pre.sort_values(by='preds',ascending=False)
    
    auc = metrics.roc_auc_score(pre.label,pre.preds)

    pre.preds=pre.preds.map(lambda x: 1 if x>=0.5 else 0)

    f1 = metrics.f1_score(pre.label,pre.preds)
    
    
    res = 0.6*auc +0.4*f1
    
    return 'res',res,True
    


In [87]:
lgb.cv(lgb_params,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=5,num_boost_round=10000,nfold=3,metrics=['evalMetric'])

[5]	cv_agg's res: 0.692042 + 0.00509468
[10]	cv_agg's res: 0.691769 + 0.0050846
[15]	cv_agg's res: 0.692043 + 0.00509344
[20]	cv_agg's res: 0.692046 + 0.00509686
[25]	cv_agg's res: 0.691766 + 0.00507752
[30]	cv_agg's res: 0.692042 + 0.00508982
[35]	cv_agg's res: 0.691783 + 0.00509854
[40]	cv_agg's res: 0.691775 + 0.00508772
[45]	cv_agg's res: 0.691775 + 0.00508772
[50]	cv_agg's res: 0.691788 + 0.00509606
[55]	cv_agg's res: 0.691971 + 0.00508164
[60]	cv_agg's res: 0.691978 + 0.00507694
[65]	cv_agg's res: 0.691987 + 0.00506596
[70]	cv_agg's res: 0.69218 + 0.00486049
[75]	cv_agg's res: 0.697512 + 0.00906216
[80]	cv_agg's res: 0.697518 + 0.00905482
[85]	cv_agg's res: 0.697533 + 0.00903588
[90]	cv_agg's res: 0.697543 + 0.00904813
[95]	cv_agg's res: 0.697543 + 0.00904813
[100]	cv_agg's res: 0.697549 + 0.00905513


{'res-mean': [0.677842720970808, 0.7006012348568259],
 'res-stdv': [0.0037983771228491774, 0.007885217832597307]}

In [88]:
model =lgb.train(lgb_params,dtrain,feval=evalMetric,verbose_eval=5,num_boost_round=300,valid_sets=[dtrain])

[5]	training's res: 0.70959
[10]	training's res: 0.709606
[15]	training's res: 0.70958
[20]	training's res: 0.709592
[25]	training's res: 0.709592
[30]	training's res: 0.709587
[35]	training's res: 0.709591
[40]	training's res: 0.709592
[45]	training's res: 0.709617
[50]	training's res: 0.709651
[55]	training's res: 0.709649
[60]	training's res: 0.709689
[65]	training's res: 0.710013
[70]	training's res: 0.710029
[75]	training's res: 0.710029
[80]	training's res: 0.710029
[85]	training's res: 0.710029
[90]	training's res: 0.710034
[95]	training's res: 0.710034
[100]	training's res: 0.710062
[105]	training's res: 0.710062
[110]	training's res: 0.710062
[115]	training's res: 0.710062
[120]	training's res: 0.710062
[125]	training's res: 0.710061
[130]	training's res: 0.710061
[135]	training's res: 0.710061
[140]	training's res: 0.710061
[145]	training's res: 0.710061
[150]	training's res: 0.710061
[155]	training's res: 0.710061
[160]	training's res: 0.710061
[165]	training's res: 0.710061

In [89]:
pred=model.predict(x_test)
pred

array([0.08033013, 0.50968402, 0.08033013, ..., 0.08033013, 0.08033013,
       0.08033013])

In [90]:
res =pd.DataFrame({'uid':test.uid,'label':pred})
res=res.sort_values(by='label',ascending=False)
res.label=res.label.map(lambda x: 1 if x>=0.5 else 0)
#res.label = res.label.map(lambda x: int(x))

In [91]:
res.to_csv('./result/lgb-stack-baseline.csv',index=False,header=False,sep=',',columns=['uid','label'])